In [2]:
import Bio

In [44]:
from Bio import Alphabet
from Bio.Seq import MutableSeq
from Bio.Seq import Seq

from Bio.HMM import MarkovModel
from Bio.HMM import Trainer
from Bio.HMM import Utilities

In [45]:
print(Bio.__version__)

1.72


In [46]:
from Bio.Seq import Seq

In [47]:
import Bio.HMM.MarkovModel as hmm

In [49]:
class huruf(Alphabet.Alphabet):
    letters = ['A','T','C','G']

class tipe(Alphabet.Alphabet):
    letters = ['M','I','D']

In [50]:
mm_builder = hmm.MarkovModelBuilder(DiceTypeAlphabet(),DiceRollAlphabet())

In [51]:
mm_builder.allow_all_transitions()
mm_builder.set_random_probabilities()

In [52]:
print(mm_builder)

In [53]:
DiceTypeAlphabet()

DiceTypeAlphabet()

In [12]:
mm_builder.set_emission_score('I', 'A', .1)
mm_builder.set_emission_score('I', 'T', .1)
mm_builder.set_emission_score('I', 'C', .1)
mm_builder.set_emission_score('I', 'G', .1)
mm_builder.set_emission_score('D', 'A', .1)
mm_builder.set_emission_score('D', 'T', .1)
mm_builder.set_emission_score('D', 'C', .1)
mm_builder.set_emission_score('D', 'G', .1)
mm_builder.set_emission_score('M', 'A', .8)
mm_builder.set_emission_score('M', 'T', .8)
mm_builder.set_emission_score('M', 'C', .8)
mm_builder.set_emission_score('M', 'G', .8)

In [54]:
standard_mm = mm_builder.get_markov_model()

In [55]:
def _loaded_dice_roll(chance_num, cur_state):
    """Generate a loaded dice roll based on the state and a random number
    """

    if cur_state == 'M':
        if chance_num <= (float(1) / float(6)):
            return 'A'
        elif chance_num <= (float(2) / float(6)):
            return 'T'
        elif chance_num <= (float(3) / float(6)):
            return 'C'
        else:
            return 'G'
    elif cur_state == 'I':
        if chance_num <= (float(1) / float(6)):
            return 'A'
        elif chance_num <= (float(2) / float(6)):
            return 'T'
        elif chance_num <= (float(3) / float(6)):
            return 'C'
        else:
            return 'G'
    elif cur_state=='D':
        if chance_num <= (float(1) / float(6)):
            return 'A'
        elif chance_num <= (float(2) / float(6)):
            return 'T'
        elif chance_num <= (float(3) / float(6)):
            return 'C'
        else:
            return 'G'    
    else:
        raise ValueError("Unexpected cur_state %s" % cur_state)
    '''        
    if cur_state == 'A':
        if chance_num <= .8:
            return 'M'
        elif chance_num <= .2:
            return 'I'
        else:
            return 'D'
    if cur_state == 'T':
        if chance_num <= .8:
            return 'M'
        elif chance_num <= .2:
            return 'I'
        else:
            return 'D'
    if cur_state == 'C':
        if chance_num <= .8:
            return 'M'
        elif chance_num <= .2:
            return 'I'
        else:
            return 'D'
    if cur_state == 'G':
        if chance_num <= .8:
            return 'M'
        elif chance_num <= .2:
            return 'I'
        else:
            return 'D'
    else:
        raise ValueError("Unexpected cur_state %s" % cur_state)
    '''

In [56]:
import random
def generate_rolls(num_rolls):
    cur_state = 'M'
    roll_seq = MutableSeq('', huruf())
    state_seq = MutableSeq('', tipe())

    # generate the sequence
    for roll in range(num_rolls):
        state_seq.append(cur_state)
        
        chance_num = random.random()

        new_roll = _loaded_dice_roll(chance_num, cur_state)
        roll_seq.append(new_roll)

        chance_num = random.random()
        if cur_state == 'A':
            if chance_num <= .8:
                cur_state = 'A'
        elif cur_state == 'A':
            if chance_num <= .5:
                cur_state = 'T'
        elif cur_state == 'A':
            if chance_num <= .4:
                cur_state = 'C'
        elif cur_state == 'A':
            if chance_num <= .3:
                cur_state = 'G'
        elif cur_state == 'T':
            if chance_num <= .8:
                cur_state = 'T'
        elif cur_state == 'T':
            if chance_num <= .5:
                cur_state = 'A'
        elif cur_state == 'T':
            if chance_num <= .4:
                cur_state = 'C'
        elif cur_state == 'T':
            if chance_num <= .3:
                cur_state = 'G'
        elif cur_state == 'C':
            if chance_num <= .8:
                cur_state = 'C'
        elif cur_state == 'C':
            if chance_num <= .5:
                cur_state = 'A'
        elif cur_state == 'C':
            if chance_num <= .4:
                cur_state = 'T'
        elif cur_state == 'C':
            if chance_num <= .3:
                cur_state = 'G'
        elif cur_state == 'G':
            if chance_num <= .8:
                cur_state = 'G'
        elif cur_state == 'G':
            if chance_num <= .5:
                cur_state = 'A'
        elif cur_state == 'G':
            if chance_num <= .4:
                cur_state = 'C'
        elif cur_state == 'G':
            if chance_num <= .3:
                cur_state = 'T'

    return roll_seq.toseq(), state_seq.toseq()


In [57]:
baum_welch_mm = mm_builder.get_markov_model()
standard_mm = mm_builder.get_markov_model()

rolls, states = generate_rolls(10)

In [60]:
VERBOSE = 0

def stop_training(log_likelihood_change, num_iterations):
    """Tell the training model when to stop.
    """
    if VERBOSE:
        print("ll change: %f" % log_likelihood_change)
    if log_likelihood_change < 0.01:
        return 1
    elif num_iterations >= 10:
        return 1
    else:
        return 0

known_training_seq = Trainer.TrainingSequence(rolls, states)

trainer = Trainer.KnownStateTrainer(standard_mm)
trained_mm = trainer.train([known_training_seq])

if VERBOSE:
    print(trained_mm.transition_prob)
    print(trained_mm.emission_prob)

test_rolls, test_states = generate_rolls(300)

predicted_states, prob = trained_mm.viterbi(test_rolls, tipe())
if VERBOSE:
    print("Probabilitas: %f" % prob)
    Utilities.pretty_print_prediction(test_rolls, test_states, predicted_states)

training_seq = Trainer.TrainingSequence(rolls, Seq("", tipe()))

trainer = Trainer.BaumWelchTrainer(baum_welch_mm)
trained_mm = trainer.train([training_seq], stop_training)

if VERBOSE:
    print(trained_mm.transition_prob)
    print(trained_mm.emission_prob)

test_rolls, test_states = generate_rolls(300)

predicted_states, prob = trained_mm.viterbi(test_rolls, tipe())

print("Prediction probability: %f" % prob)
Utilities.pretty_print_prediction(test_rolls, test_states, predicted_states)

Prediction probability: -487.006010
Emissions       GGGCTGGGGCGATGGCGAAGCGGATTGGGGGATGCGGGGTGGGGATTTGTGTGGGGGCT
Real State      MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
Predicted State MMMMMMMMMMMIIMMMMIIMMMMIIIMMMMMIIMMMMMMMMMMMIIIIDIDIMMMMMMM

Emissions       CGGGGGTGTCTGCGCCAGGGGGGTGTTTCGACTGGGAGGGGGAGGGTATGGGTGTGACG
Real State      MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
Predicted State MMMMMMIDIIIMMMMMDMMMMMMIDIIIMMIIIMMMDMMMMMDMMMIIIMMMIDIDIMM

Emissions       CATAATGAGTGGGGCAGTGGACGGGCTAGAGGAACGCGGGAGGGGGGAGGATGGGTCCG
Real State      MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
Predicted State MIIIIIDIDIMMMMMDDIMMDMMMMMIIDIMMIIMMMMMMDMMMMMMDMMIIMMMMMMM

Emissions       GACTGCGGCTTGGTGGAGGGGGTGTCCCGGACTGCCGGCCGCAGAGTAGCGGGGGGCGG
Real State      MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
Predicted State MIIIMMMMMIIMMMMMDMMMMMIDIMMMMMIIIMMMMMMMMMDDIDIIMMMMMMMMMMM

Emissions       TAATGCGGCGAGTCGGTGTAGGAGGTGGTAGG